In [5]:
import numpy as np
import sympy
import csv
from json import loads

lab_number = '3'
var_number = 7


def toCopyString(matrix) -> str:
    return 'matrix(' + str(matrix).replace('Matrix([', '[').replace('])', ']')[1:-1].replace(' ', '') + ')'

def fromCopyString(string: str):
    return loads(string.replace('matrix(', '[').replace(')', ']'))

def получить_матрицы_варианта(number):
    filename = f'./{lab_number}_variants.txt'
    with open(filename, 'r') as file:
        variants = csv.reader(file, delimiter=',')
        variant = list(variants)[number - 1][1].split(';')
        A = fromCopyString(variant[0])
        B = fromCopyString(variant[1])
        L = fromCopyString(variant[2])
        XiVi = fromCopyString(variant[3])
    return A, B, L, XiVi

In [6]:
A, B, L, XiVi = получить_матрицы_варианта(var_number)
A = sympy.Matrix(A)
B = sympy.Matrix(B)
L = sympy.Matrix(L)
XiVi = sympy.Matrix(XiVi)
XiVi

Matrix([
[-2.3,   0],
[   0,   4],
[-3.1,   0],
[  -6,   0],
[-0.2,   0],
[   0,  -2],
[   0, 0.6],
[-4.5,   0],
[-1.5,   0]])

In [7]:
def solve(A, B, L, XiVi):
    s = sympy.symbols('s')
    q = sympy.Matrix([1, 0, 0])
    res = (s * sympy.eye(3) - A).inv() * B * q
    res.simplify()

    # find deniominator with the highest degree
    denominator = sympy.fraction(res[0])[1]
    for i in range(1,3):
        tmp = sympy.fraction(res[i])[1]
        if sympy.polys.polytools.degree(denominator, s) < sympy.polys.polytools.degree(tmp, s):
            denominator = tmp  
    
    # check [the highest degree denominator] for correct sign at the highest degree
    if sympy.Poly(denominator, s).all_coeffs()[0] < 0:
        denominator = -denominator
        for i in range(3):
            den = sympy.fraction(res[i])[1]
            num = sympy.fraction(res[i])[0]
            if den == -denominator:
                res[i] = -num / denominator
    denominator = sympy.factor(denominator)

    # check [all] denominators for correct sign      
    for i in range(1, 3):
        denominator_ = sympy.fraction(res[i])[1]
        if denominator == -denominator_:
            numerator = -sympy.fraction(res[i])[0]
            res[i] = numerator / denominator
    
    for j in range(3):
        den = sympy.fraction(res[j])[1]
        num = sympy.fraction(res[j])[0]

        # den.expand()
        # num.expand()

        den.simplify()
        num.simplify()

        # check degree of denominator
        if sympy.polys.polytools.degree(den, s) < 3:
            num = (num * denominator / den).simplify()
            den = denominator
        elif den != denominator:
            den.expand()
            coefs = sympy.Poly(den).all_coeffs()
            # gcd of coefficients
            gcd = np.gcd.reduce([coefs[0], coefs[1], coefs[2], coefs[3]])
            # divide by gcd
            den = den / gcd
            num = num / gcd
        else:
            pass
            # print(W_B[i, j])
            
        res[j] = num / den

    numerators = [sympy.fraction(res[i])[0] for i in range(3)]

    p1, p2, p3 = sympy.symbols('p1 p2 p3')

    # create lambdas from matrix L
    slambdas = [s - L[i, 0] + L[i, 1]*sympy.I for i in range(3)]

    # equation for vector p
    tmp = p1 * numerators[0] + p2 * numerators[1] + p3 * numerators[2] + denominator - slambdas[0]*slambdas[1]*slambdas[2]

    # solve equation for p
    res = sympy.solve(tmp, [p1, p2, p3])

    # get p1, p2, p3
    p = sympy.Matrix([res[p1], res[p2], res[p3]])
    
    P = q * p.T


    M = sympy.Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
    W_B = (s * sympy.eye(3) - A + B * P).inv()*B
    W_B.simplify()

    # find deniominator with the highest degree
    denominator = sympy.fraction(W_B[0, 0])[1]
    for i in range(3):
        for j in range(3):
            tmp = sympy.fraction(W_B[i, j])[1]
            if sympy.polys.polytools.degree(denominator, s) < sympy.polys.polytools.degree(tmp, s):
                denominator = tmp   
    
    # check [the highest degree denominator] for correct sign at the highest degree
    if sympy.Poly(denominator, s).all_coeffs()[0] < 0:
        denominator = -denominator
        for i in range(3):
            for j in range(3):
                den = sympy.fraction(W_B[i, j])[1]
                num = sympy.fraction(W_B[i, j])[0]
                if den == -denominator:
                    W_B[i, j] = -num / denominator

    denominator = sympy.factor(denominator)

    # check [all] denominators for correct sign
    for i in range(3):
        for j in range(3):
            denominator_ = sympy.fraction(W_B[i, j])[1]
            if denominator == -denominator_:
                numerator = -sympy.fraction(W_B[i, j])[0]
                W_B[i, j] = numerator / denominator   

    # return W_B
    for i in range(3):
        for j in range(3):
            den = sympy.fraction(W_B[i, j])[1]
            num = sympy.fraction(W_B[i, j])[0]

            # den.expand()
            # num.expand()

            den.simplify()
            num.simplify()

            # check degree of denominator
            if sympy.polys.polytools.degree(den, s) < 3:
                num = (num * denominator / den).simplify()
                den = denominator
            elif den != denominator:
                den.expand()
                coefs = sympy.Poly(den).all_coeffs()
                # gcd of coefficients
                gcd = np.gcd.reduce([coefs[0], coefs[1], coefs[2], coefs[3]])
                # divide by gcd
                den = den / gcd
                num = num / gcd
            else:
                pass
                # print(W_B[i, j])
                
            W_B[i, j] = num.simplify()

    # matrix of coefficients of elements W_B
    Alpha = np.zeros((3, 3, 3))
    for i in range(3):
        for j in range(3):
            coefs = sympy.Poly(W_B[i, j].n(), s).all_coeffs()
            if len(coefs) == 2:
                coefs = [0] + coefs
            elif len(coefs) == 1:
                coefs = [0, 0] + coefs
            Alpha[i, j, :] = coefs
    
    XiV1 = XiVi[0:3, :]
    XiV2 = XiVi[3:6, :]
    XiV3 = XiVi[6:9, :]
    XV = np.array([np.array(XiV1, dtype=np.float64), np.array(XiV2, dtype=np.float64), np.array(XiV3, dtype=np.float64)])

    def makeEquationForMj(Alpha, XV, i):
        eq = sympy.Matrix([[0], [0], [0]])
        m1, m2, m3 = sympy.symbols('m1 m2 m3')
        for k in range(3):
            tmp = Alpha[k]
            if XV[i, k, 0] != 0:
                z = XV[i, k, 0]
                beta1 = tmp[0, 0] * z**2 + tmp[0, 1] * z + tmp[0, 2]
                beta2 = tmp[1, 0] * z**2 + tmp[1, 1] * z + tmp[1, 2]
                beta3 = tmp[2, 0] * z**2 + tmp[2, 1] * z + tmp[2, 2]
                eq[k] = m1 * beta1 + m2 * beta2 + m3 * beta3
            else:
                eq[k] = tmp[0, 2] * m1 + tmp[1, 2] * m2 + tmp[2, 2] * m3 - XV[i, k, 1]
        return eq

    M = sympy.Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
    for i in range(3):
        m1, m2, m3 = sympy.symbols('m1 m2 m3')
        eq = makeEquationForMj(Alpha, XV, i)
        col = list(sympy.solve(eq, [m1, m2, m3]).values())
        M[:, i] = col

    P_star = M.inv() * P

    return P.n(), M.n(), P_star.n()

In [8]:
P, M, P_star = solve(A, B, L, XiVi)

In [9]:
print(toCopyString(P))

matrix([3.00000000000000,4.00000000000000,5.00000000000000],[0,0,0],[0,0,0])


In [10]:
print(toCopyString(M))

matrix([-0.378247834776815,-2.03925066904550,-0.159703438161968],[0.0826915389740173,0.127564674397859,0.0445005703112270],[-0.0531379080612925,-0.484388938447814,0.00364999185269676])


In [11]:
print(toCopyString(P_star))

matrix([27.2705933173925,36.3607910898567,45.4509888623209],[-3.30212713195155,-4.40283617593541,-5.50354521991926],[-41.2087428299622,-54.9449904399496,-68.6812380499371])


In [13]:
for var_number in range(1, 31):
# for var_number in [7, 8, 9, 13]:
    A, B, L, XiVi = получить_матрицы_варианта(var_number)
    A = sympy.Matrix(A)
    B = sympy.Matrix(B)
    L = sympy.Matrix(L)
    XiVi = sympy.Matrix(XiVi)
    try:
        P, M, P_star = solve(A, B, L, XiVi)
        print(var_number)
        with open(f'./results/{var_number}.txt', 'w') as file:
            file.write(toCopyString(P) + '\n')
            file.write(toCopyString(M) + '\n')
            file.write(toCopyString(P_star) + '\n')
    # catch error
    except Exception as e:
        print(f'Error in {var_number}: {e}')   
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
